# Imersão Dev Agentes de IA - Alura e Google Gemini



## Aula 01 - Classificação de intenções com IA

In [17]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

In [18]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI


GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [19]:
# temperatura entre 0 e 1, 1 sendo criativo

In [20]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperatura=0.8,
    api_key=GOOGLE_API_KEY
)

In [21]:
resp_test = llm.invoke("Quem é você?")
print(resp_test.content)

Eu sou um modelo de linguagem grande, treinado pelo Google.


In [22]:
# Qual o objetivo do Agente?

In [23]:
TRIAGEM_PROMPT = (
    "Você é um gerador de README para perfil do GitHub. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_COMPLETAR" | "PEDIR_INFO" | "GERAR_BIO",\n'
    '  "carreira_academica": "ESTUDANTE" | "FORMADO/A" | "PÓS-GRADUADO/A",\n'
    '  "experiencia_profissional": "ESTAGIÁRIO/A" | "JÚNIOR" | "PLENO" | "SÊNIOR | "ESPECIALISTA",\n'
    '  "campos_faltantes: ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_COMPLETAR**: Perfil completo da pessoa, contendo nome, função, senioridade, hard skills e soft skills (Ex: "Meu nome é Maria, sou Especialista em TI Sênior, programadora javascript e autodidata", "Meu nome é José, sou estudante buscando estágio em TI, sei HMTL, CSS e JS").\n'
    '- **PEDIR_INFO**: Mensagem de apresentação vaga ou que faltam informações para identificar o perfil (Ex: "Sou estudante", "Trabalho com TI").\n'
    '- **GERAR_BIO**: Solicitação para criar Bio ou Perfil do GitHub com as informações da mensagem de forma completa e precisa."'
)

In [24]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao: Literal["AUTO_COMPLETAR", "PEDIR_INFO", "GERAR_BIO"]
    carreira_academica: Literal["ESTUDANTE", "FORMADO/A", "PÓS-GRADUADO/A"]
    experiencia_profissional: Literal["ESTAGIÁRIO/A", "JÚNIOR", "PLENO", "SÊNIOR", "ESPECIALISTA"]
    campos_faltantes: List[str] = Field(default_factory=list)

In [25]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [26]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()

In [27]:
testes = ["Meu nome é Ana Maria sou Especialista de TI, Formada e conclui a Pós, sou Analista nível Pleno", "Meu Nome é Maria, sou estudante e estagiária"]

In [28]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Meu nome é Ana Maria sou Especialista de TI, Formada e conclui a Pós, sou Analista nível Pleno
 -> Resposta: {'decisao': 'AUTO_COMPLETAR', 'carreira_academica': 'PÓS-GRADUADO/A', 'experiencia_profissional': 'PLENO', 'campos_faltantes': ['hard skills', 'soft skills']}

Pergunta: Meu Nome é Maria, sou estudante e estagiária
 -> Resposta: {'decisao': 'AUTO_COMPLETAR', 'carreira_academica': 'ESTUDANTE', 'experiencia_profissional': 'ESTAGIÁRIO/A', 'campos_faltantes': []}



## Aula 02 - Construindo a base de conhecimento com RAG

In [16]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

In [30]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

docs = []

for n in Path("/content/").glob("*.pdf"):
  try:
    loader = PyMuPDFLoader(str(n))
    docs.extend(loader.load())
    print(f"Carregado arquivo {n.name} com sucesso")
  except Exception as e:
    print(f"Erro ao carregar {n.name}: e")

print(f"Total de documentos carregados: {len(docs)}")

Carregado arquivo doc.pdf com sucesso
Total de documentos carregados: 1


In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

chunks = splitter.split_documents(docs)

In [32]:
for chunk in chunks:
    print(chunk)
    print("------------------------------------")

page_content='# Ana Maria - Portfólio de Projetos 
 
Sou **Ana Maria**, uma **Cientista da Computação** com experiência como **Analista 
de TI** e focada em **desenvolvimento web**. Este espaço é um portfólio dos meus 
projetos e contribuições. 
 
##   Destaques' metadata={'producer': 'Microsoft® Word para Microsoft 365', 'creator': 'Microsoft® Word para Microsoft 365', 'creationdate': '2025-09-14T14:22:46-03:00', 'source': '/content/doc.pdf', 'file_path': '/content/doc.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': 'Ana Maria Araujo da Silva', 'subject': '', 'keywords': '', 'moddate': '2025-09-14T14:22:46-03:00', 'trapped': '', 'modDate': "D:20250914142246-03'00'", 'creationDate': "D:20250914142246-03'00'", 'page': 0}
------------------------------------
page_content='##   Destaques 
 
*   [Nome do Projeto 1](link-para-o-projeto-1): Uma aplicação web para gestão de tarefas, 
desenvolvida com React e Node.js. 
*   [Nome do Projeto 2](link-para-o-projeto-2): Script 

In [33]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key=GOOGLE_API_KEY
)

In [34]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold":0.3, "k": 4})

In [35]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente Gerador de README para o Perfil do GitHub. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [36]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [37]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
    docs_relacionados = retriever.invoke(pergunta)

    if not docs_relacionados:
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    answer = document_chain.invoke({"input": pergunta,
                                    "context": docs_relacionados})

    txt = (answer or "").strip()

    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    return {"answer": txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado": True}

In [39]:
testes = ["Sou cientista da computação e analista de TI"]

In [40]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("CITAÇÕES:")
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------")

PERGUNTA: Sou cientista da computação e analista de TI
RESPOSTA: Sim, você é Cientista da Computação e Analista de TI.
CITAÇÕES:
 - Documento: doc.pdf, Página: 1
   Trecho: # Ana Maria - Portfólio de Projetos Sou **Ana Maria**, uma **Cientista da Computação** com experiência como **Analista de TI** e focada em **desenvolvimento web**. Este espaço é um 
------------------------------------
